In [1]:
import pandas as pd
import numpy as np

# Lendo o dataset

In [2]:
df = pd.read_csv(r"C:\Users\gabri\Desktop\TCC\Data\casosgravescovid.csv",index_col=None, header=0,sep=';',decimal=',')

In [6]:
df['sexo'].unique()

array(['Masculino', 'Feminino', 'Ignorado'], dtype=object)

In [7]:
df['classificacao_final'].unique()

array(['DESCARTADO', 'INCONCLUSIVO', 'CONFIRMADO', 'EM ANÁLISE'],
      dtype=object)

In [11]:
def categorizing_sexo(sexo):
            if sexo == 'Masculino':
                sexo = 1
            elif sexo == 'Feminino':
                sexo = 0
            elif sexo == 'Ignorado':
                sexo = 0
            return sexo
        
df['sexo'] = df['sexo'].apply(categorizing_sexo)

In [13]:
df['evolucao'].unique()

array(['ISOLAMENTO DOMICILIAR', 'INTERNADO LEITO DE ISOLAMENTO',
       'RECUPERADO', 'ÓBITO', nan, 'INTERNADO UTI'], dtype=object)

In [34]:
list_months = ['IGN', '1 meses', '1 mês', '0 meses','1 mes', '2 meses','3 meses','4 meses','5 meses', '6 meses', '7 meses','8 meses', 
                '9 meses','10 meses', '11 meses', '12 meses']

for i in range(len(df)):
    if df['idade'][i] in list_months:
        df['idade'][i] = 0
    else:
        df['idade'][i] = int(df['idade'][i])

In [49]:
df['idade'] = df['idade'].astype(int)

In [73]:
from datetime import datetime
df['data_inicio_sintomas'] = df['data_inicio_sintomas'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [81]:
df['data_obito'].fillna('', inplace = True)
df['data_obito'] = df['data_obito'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

ValueError: time data '' does not match format '%Y-%m-%d'

# Pré-processamento

## Tratamento dos Dados

### Excluindo dados não relevantes

In [59]:
df = df.drop(columns=["etnia","municipio_residencia"])

### Tratando dados ausentes

In [60]:
df = df.fillna("Não informado")

### Lendo Arquivo com sintomas

In [37]:
from csv import reader

with open(r'C:\Users\gabri\Desktop\TCC\Data\sintomas.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = reader(csv_file, delimiter = ';')
    lista_sint = list(csv_reader)
    
    lista_sint = list(lista_sint[0])
    lista_sint[0] = "nao informado"
    
    print(lista_sint)

['nao informado', 'coriza', 'astenia', 'cefaleia', 'adinamia', 'mialgia', 'dor no corpo', 'taquipneia', 'diarreia', 'nausea', 'vomitos', 'alteracao da ausculta pulmonar', 'moleza', 'irritacao nos olhos', 'dor abdominal', 'aperto toracico', 'irritabilidade', 'confusao', 'dor de ouvido', 'dor de cabeca', 'tosse', 'distúrbios olfativos', 'distúrbios gustativos', 'dor de garganta', 'coriza', 'febre', 'dispneia', 'hiperemia Faríngea', 'pigarro', 'epistaxe', 'espirro', 'dor de cabeça', 'assintomático']


### Tratando a feature sintomas e outros sintomas

Separando as strings em listas

In [38]:
lista_sintomas = []
lista_outros_sintomas = []

for i in range(len(df)):
    #esse for corre todo o DF e separa a string em lista de sintomas
    
    sintomas = df["sintomas"][i]
    df["sintomas"][i] = sintomas.split(',')
    
    outros_sintomas = df["outros_sintomas"][i]
    df["outros_sintomas"][i] = outros_sintomas.split(',')
    
    #criando a lista temporária de sintomas por conta da necessidade de pré-processar
    sintomas_temp = []
    outros_sintomas_temp = []
    
    #esse for percorre a lista criada, aplica a função strip e lower 
    for sintoma in df["sintomas"][i]:
        sintoma = sintoma.strip().lower()
        sintomas_temp.append(sintoma)
        
        '''
        #esse for utiliza o csv dos sintomas para separar as substrings
        for sint in lista_sint:
            resp = sint in sintoma
            if resp == True:
                sintomas_temp.append(sint)
         '''       
        #atualiza o DF com a lista pré-processada 
        df["sintomas"][i] = sintomas_temp
        
        #esse for corre a lista sintomas_tempo e ADD os sintomas novos a lista_sintomas
        for sintoma in sintomas_temp:
            if sintoma not in lista_sintomas:
                lista_sintomas.append(sintoma)
                
    #REPETINDO O CÓDIGO PRA FEATURE OUTRO_SINTOMA
    
    #esse for percorre a lista criada, aplica a função strip e lower
    for outro_sintoma in df["outros_sintomas"][i]:
        outro_sintoma = outro_sintoma.strip().lower()
        
        outros_sintomas_temp.append(outro_sintoma)
        
        '''
        #esse for utiliza o csv dos sintomas para separar as substrings
        for outro_sint in lista_sint:
            resp = outro_sint in outro_sintoma
            if resp == True:
                outros_sintomas_temp.append(outro_sint)
        '''     
        
        #atualiza o DF com a lista pré-processada 
        df["outros_sintomas"][i] = outros_sintomas_temp
        
        #esse for corre a lista sintomas_tempo e ADD os sintomas novos a lista_sintomas
        for outro_sintoma in outros_sintomas_temp:
            if outro_sintoma not in lista_outros_sintomas:
                lista_outros_sintomas.append(outro_sintoma)

In [39]:
lista_sintomas

['febre',
 'tosse',
 'mialgia',
 'alteração/perda de olfato e/ou paladar',
 'dispneia',
 'saturação o2 < 95',
 'não informado',
 'diarreia',
 'dor de garganta',
 'coriza / congestão nasal',
 'erupções cutâneas',
 'cefaleia',
 'vômito',
 'cansaço/fadiga',
 'desconforto respiratório / aperto torácico',
 'náusea',
 'batimento asa de nariz',
 'tiragem intercostal',
 'cianose',
 'edema mãos/pés']

In [40]:
lista_outros_sintomas

['não informado',
 'astenia',
 'digeusia',
 'hiposmia',
 'dor epigastrica',
 'falta de ar',
 'espirro',
 'anorexia',
 'dor abdominal',
 'dor lombar',
 'hematemese',
 'rinorreia astenia',
 'disuria',
 'calafrios',
 'cefaleia',
 'mal estar',
 'prostração',
 'fraqueza',
 'coriza nasal',
 'adinamia',
 'ageusia',
 'dor toracica',
 'taquicardia',
 'disgeusia',
 'falência respiratória. sonolência',
 'sudorese',
 'inapetência',
 'anosmia',
 'tontura',
 'etilista cronico',
 'dor pleurítica',
 'sonolência',
 'desorientação',
 'dor precordial',
 'pigarro',
 'espirros',
 'moleza no corpo',
 'rinorreia',
 'escarro com raios de sangue',
 'queda do estado geral',
 'mialgia',
 'congestão nasal',
 'perda de peso ponderal',
 'sudorese+hipotensao',
 'dor nas costas',
 'crise convulsiva',
 'hemoptise',
 'obstrucao nazal',
 'astenia e hiporexia',
 'tremores em dimidio esquerdo',
 'desorientação.',
 'dor torácica',
 'ortopneia',
 'palidez',
 'mialgia e astenia',
 'astenia e mialgia',
 'anorexia.',
 'dorsalg

### Tratando as doenças

In [61]:
lista_doencas = []
lista_outras_doencas = []

for i in range(len(df)):
    #esse for corre todo o DF e separa a string em lista de sintomas
    
    doencas = df["doencas_preexistentes"][i]
    df["doencas_preexistentes"][i] = doencas.split(',')
    
    outras_doencas = df["outras_doencas_preexistentes "][i]
    df["outras_doencas_preexistentes "][i] = outras_doencas.split(',')
    
    #criando a lista temporária de sintomas por conta da necessidade de pré-processar
    doencas_temp = []
    outras_doencas_temp = []
    
    #esse for percorre a lista criada, aplica a função strip e lower 
    for doenca in df["doencas_preexistentes"][i]:
        doenca = doenca.strip().lower()
        doencas_temp.append(doenca)
        
        '''
        #esse for utiliza o csv dos sintomas para separar as substrings
        for sint in lista_sint:
            resp = sint in sintoma
            if resp == True:
                sintomas_temp.append(sint)
         '''       
        #atualiza o DF com a lista pré-processada 
        df["doencas_preexistentes"][i] = doencas_temp
        
        #esse for corre a lista sintomas_tempo e ADD os sintomas novos a lista_sintomas
        for doenca in doencas_temp:
            if doenca not in lista_doencas:
                lista_doencas.append(doenca)
                
    #REPETINDO O CÓDIGO PRA FEATURE OUTRO_SINTOMA
    
    #esse for percorre a lista criada, aplica a função strip e lower
    for outra_doenca in df["outras_doencas_preexistentes "][i]:
        outra_doenca = outra_doenca.strip().lower()
        
        outras_doencas_temp.append(outra_doenca)
        
        '''
        #esse for utiliza o csv dos sintomas para separar as substrings
        for outro_sint in lista_sint:
            resp = outro_sint in outro_sintoma
            if resp == True:
                outros_sintomas_temp.append(outro_sint)
        '''     
        
        #atualiza o DF com a lista pré-processada 
        df["outras_doencas_preexistentes "][i] = outras_doencas_temp
        
        #esse for corre a lista sintomas_tempo e ADD os sintomas novos a lista_sintomas
        for outra_doenca in outras_doencas_temp:
            if outra_doenca not in lista_outras_doencas:
                lista_outras_doencas.append(outra_doenca)

In [62]:
lista_doencas

['não informado',
 'portador de doenças cromossômicas',
 'diabetes',
 'doenças cardíacas ou vasculares',
 'sobrepeso/obesidade',
 'doenças respiratórias crônicas',
 'doença hepática crônica',
 'doenças renais cronicas',
 'imunossupressão',
 'gestação',
 'doenças neurológicas',
 'neoplasias',
 'asma',
 'doenças hematológicas',
 'puérpera']

In [63]:
lista_outras_doencas

['não informado',
 'doenca neurologica',
 'neoplasias',
 'alzheimer',
 'has e avc?',
 'etilista',
 'has e drc em tsr',
 'além de ca de próstata com metástase óssea',
 'doenca endocrina (não dm)',
 'ignorado',
 'dmg',
 'pneumonia cronica',
 'ex tabagista',
 'prematuridade',
 'tabagismo',
 'ca pulmao',
 'has',
 'dlp',
 'drc',
 'adenocarcinoma de pulmão',
 'fa cronica.',
 'câncer da face',
 'pele',
 'seq. avc',
 'has / tabagismo / cardiopatia',
 'neoplasia',
 'câncer',
 'has/ rim unico',
 'asma',
 'obesidade',
 'vitiligo',
 'glaucoma',
 'tabagista',
 'antecendente ca mama',
 'alzaimer',
 'dhc secundária a hemocromatose- passado de hepatocarcinoma tratado apenas com quimioembolização  - hipotireoidismo - aneurisma em mie (sic)',
 'hipotireoidismo e dlp',
 'anemia + dislipidemia + tabagista + tb pulmonar',
 'ave',
 'neuroblasfoma',
 'hipertensão / etilismo crônico',
 'doença hematológica cronica',
 'leucemia linfóide aguda b',
 'tuberculose pulmonar.',
 'etilista e tabagista',
 'etilismo',


#### Categorizar essas doenças!